In [1]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. 문서 내용 읽고 분할 ########################################
loader = Docx2txtLoader('law_1.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

# 2. 임베딩 -> 벡터 데이터베이스에 저장 ######################################

## 2-1 환경변수 읽어오기
load_dotenv()

## 2-2. 임베딩 모델 지정 
embedding = OpenAIEmbeddings(model='text-embedding-3-large')


## 2-3. 벡터 데이터베이스에 저장 
## [방법 1] in memory
# database = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
# )



In [2]:


# ##[방법 2] 로컬에 저장
# database = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     persist_directory='./chroma',
#     collection_name='chroma-law',
# )

# database


In [3]:
## 로컬에 저장된 임베딩 데이터 가져오기
from langchain_chroma import Chroma

database = Chroma(
    collection_name='chroma-law',
    persist_directory='./chroma',
    embedding_function=embedding,
)

database

In [4]:


## 3. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색 ############################
## 3-1. 사용자 질문
query = '전세사기피해자로 인정받기 위한 조건은? '

## 3-2. 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query, k=3)

## 4. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 #############################
## 4-1 프롬프트 작성 
prompt = '''
[identity]
- 당신은 전세사기피해 법률 전문가입니다.
-[context]를 참고하여 사용자의 질문에 답변해주세요.

[content]
{retrieved_docs}

Question: {query}

'''
## 4-2. 프롬프트 변수에 값 설정
formatted_prompt = prompt.format(
    retrieved_docs=retrieved_docs,
    query=query
)


from langchain_openai import ChatOpenAI

## 4-3. LLM 모델 생성(chatOpenAI 인스턴스 생성)
llm = ChatOpenAI(model='gpt-4o')
## 4-4. LLM 모델에 질문과 검색된 문서를 보냄
ai_message = llm.invoke(formatted_prompt)
ai_message

AIMessage(content='전세사기피해자로 인정받기 위해서는 다음의 요건을 모두 갖추어야 합니다. 단, 경매 또는 공매 절차가 완료된 임차인의 경우에는 첫 번째 및 세 번째 요건은 제외됩니다.\n\n1. 주택임대차보호법 제3조에 따라 주택의 인도와 주민등록을 마치고, 같은 법 제3조의2 제2항에 따라 임대차계약증서상의 확정일자를 갖출 것. 임차권등기를 마친 경우 또는 민법 제303조에 따라 전세권이 설정된 경우도 포함됩니다.\n\n2. 임차인의 임차보증금이 5억원 이하일 것. 다만, 전세사기피해지원위원회가 시ㆍ도별 여건 및 피해자의 여건 등을 고려하여 임차보증금의 상한액을 2억원의 범위에서 상향 조정할 수 있습니다.\n\n3. 임대인의 파산 또는 회생절차 개시, 임차주택의 경매 또는 공매절차의 개시 등으로 인해 다수의 임차인에게 임차보증금 반환 채권의 변제를 받지 못하는 피해가 발생하였거나 발생할 것이 예상될 것.\n\n4. 임대인 등에게 수사가 개시되었거나, 임대인의 기망, 임차보증금을 반환할 능력이 없는 자에게 임차주택이 양도되었거나, 임차보증금을 반환할 능력 없이 다수의 주택을 취득ㆍ임대하여 임차보증금 반환채무를 이행하지 않을 의도가 있었다고 의심할 만한 상당한 이유가 있을 것.\n\n또한, 임차인이 임차보증금 반환을 위한 보증 또는 보험에 가입하여 보증금 전액 반환이 가능한 경우, 보증금 전액이 최우선 변제가 가능한 경우, 대항력 또는 우선변제권 행사를 통해 보증금 전액을 자력으로 회수할 수 있는 경우는 전세사기피해자로 인정받기 위한 요건에서 제외됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 458, 'prompt_tokens': 3139, 'total_tokens': 3597, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens'

In [5]:
len(retrieved_docs)

3

In [6]:
%whos

Variable                         Type                              Data/Info
----------------------------------------------------------------------------
ChatOpenAI                       ModelMetaclass                    <class 'langchain_openai.<...>_models.base.ChatOpenAI'>
Chroma                           ABCMeta                           <class 'langchain_chroma.vectorstores.Chroma'>
Docx2txtLoader                   ABCMeta                           <class 'langchain_communi<...>document.Docx2txtLoader'>
OpenAIEmbeddings                 ModelMetaclass                    <class 'langchain_openai.<...>s.base.OpenAIEmbeddings'>
RecursiveCharacterTextSplitter   ABCMeta                           <class 'langchain_text_sp<...>veCharacterTextSplitter'>
ai_message                       AIMessage                         content='전세사기피해자로 인정받기 위해<...>dio': 0, 'reasoning': 0}}
database                         Chroma                            <langchain_chroma.vectors<...>ct at 0x000001BAC59C